In [1]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
def log(message):
    print("Log: " + message)

In [3]:
def navigate_to_address_page(browser, eye_button):
    print("Log: Navigating to address page")
    browser.execute_script("arguments[0].click();", eye_button);
    # time.sleep(2)

def get_address_data(browser):
    address_obj = {}
    kls_element = WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.ID, 'processPageForm:klsId')))
    address_obj["kls_id"] = kls_element.text
    log("Starting with kls: " + address_obj["kls_id"])
    address_obj["street"] = browser.find_element("id", "processPageForm:street").text
    address_obj["house_number"] = browser.find_element("id", "processPageForm:houseNumber").text
    address_obj["postal"] = browser.find_element("id", "processPageForm:postalCode").text
    address_obj["city"] = browser.find_element("id", "processPageForm:city").text
    return address_obj["kls_id"], address_obj

def navigate_to_contact_people_tab(browser):
    log("Navigating to contact tab")
    contact_tab_button = browser.find_element("xpath", '//a[@href="#processPageForm:propertyTabView:contactData"]')
    browser.execute_script("arguments[0].click();", contact_tab_button);
    WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.ID, 'processPageForm:propertyTabView:contactDataTable_data')))

def get_contact_people_list(browser):
    contact_table = browser.find_element("id", "processPageForm:propertyTabView:contactDataTable_data")
    if "No contact persons available" in contact_table.get_attribute("outerHTML"):
        print("There is not contact people!")
        return []
    contact_peaple_rows = contact_table.find_elements('css selector', 'tr')
    contact_peaple_list = []
    for pearson_row in contact_peaple_rows:
        html_columns = pearson_row.find_elements('css selector', 'td')
        person = {}
        person["name"] = html_columns[0].find_element('css selector', 'span').text
        person["role"] = html_columns[1].find_element('css selector', 'span').text
        person["fixedline"] = html_columns[2].find_element('css selector', 'span').text
        person["mobile"] = html_columns[3].find_element('css selector', 'span').text
        person["email"] = html_columns[4].find_element('css selector', 'span').text
        person["sms"] = html_columns[5].find_element('css selector', 'span').text
        person["preferred"] = html_columns[6].find_element('css selector', 'span').text
        contact_peaple_list.append(person)
        print("Reading contact person: ", person)
    return contact_peaple_list

def navigate_to_owner_tab(browser):
    log("Navigating to owner tab")
    owner_tab_button = browser.find_element("xpath", '//a[@href="#processPageForm:propertyTabView:propertyOwner"]')
    browser.execute_script("arguments[0].click();", owner_tab_button);
    WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.ID, 'processPageForm:propertyTabView:propertyOwnerTable_data')))

def get_owners_list(browser):
    owners_list = []
    owners_table = browser.find_element("id", "processPageForm:propertyTabView:propertyOwnerTable_data")
    owners_rows = owners_table.find_elements('css selector', 'tr')
    for owner_row in owners_rows:
        html_columns = owner_row.find_elements('css selector', 'td')
        owner = {}
        owner["name"] = html_columns[0].find_element('css selector', 'span').text
        owner["email"] = html_columns[1].find_element('css selector', 'span').text
        owner["mobil"] = html_columns[2].find_element('css selector', 'span').text
        owner["linenumber"] = html_columns[3].find_element('css selector', 'span').text
        owner["postcode"] = html_columns[4].find_element('css selector', 'span').text
        owner["city"] = html_columns[5].find_element('css selector', 'span').text
        owner["street"] = html_columns[6].find_element('css selector', 'span').text
        owner["housenumber"] = html_columns[7].find_element('css selector', 'span').text
        owner["decisionmaker"] = html_columns[8].find_element('css selector', 'input').get_attribute("aria-checked")
        print("Reading owner: ", owner)
        owners_list.append(owner)
    return owners_list

def close_address_page(browser):
    log("Pressing close button of address page")
    close_button = browser.find_element('id', 'page-header-form:closePropertyDetailsPage')
    browser.execute_script("arguments[0].click();", close_button);


In [4]:
def get_eye_data(browser, eye_button):
    address_tab_data = {}
    navigate_to_address_page(browser, eye_button)
    kls_id, address_obj = get_address_data(browser)
    address_tab_data["adress_obj"] = address_obj

    navigate_to_contact_people_tab(browser)
    address_tab_data["contact_people"] = get_contact_people_list(browser)

    navigate_to_owner_tab(browser)
    address_tab_data["owners"] = get_owners_list(browser)

    close_address_page(browser)
    time.sleep(2)
    return kls_id, address_tab_data



def get_eyes_data(browser):
    kls_dicts = {}
    eyes_links = browser.find_elements("xpath", '//a[contains(@id,":viewSelectedRowItem")]')
    number_of_eyes = len(eyes_links)
    for eye_index in range(number_of_eyes):
        # refresh eys_lin
        kls_id, address_tab_data = get_eye_data(browser, eyes_links[eye_index])
        kls_dicts[kls_id] = address_tab_data
        eyes_links = browser.find_elements("xpath", '//a[contains(@id,":viewSelectedRowItem")]')
    return kls_dicts

In [5]:
def export_kls_to_excel(kls_dicts):
    klsid = []
    ort = []
    strasse = []

    rolle = []
    name = []
    person_ort = []
    person_street = []
    telefon = []
    mobile = []
    email = []
    for kls in kls_dicts.keys():
        adress_obj = kls_dicts[kls]["adress_obj"]
        contacts_list = kls_dicts[kls]["contact_people"]
        owners_list = kls_dicts[kls]["owners"]
        for contact in contacts_list:
            klsid.append(kls)
            ort.append(adress_obj["postal"] + " " + adress_obj["city"])
            strasse.append(adress_obj["street"] + " " + adress_obj["house_number"])
            
            rolle.append(contact["role"])
            name.append(contact["name"])
            person_ort.append("No Ort")
            person_street.append("No Street")
            telefon.append(contact["fixedline"])
            mobile.append(contact["mobile"])
            email.append(contact["email"])

        for owner in owners_list:
            klsid.append(kls)
            ort.append(adress_obj["postal"] + " " + adress_obj["city"])
            strasse.append(adress_obj["street"] + " " + adress_obj["house_number"])

            rolle.append("Inhaber")
            name.append(owner["name"])
            person_ort.append(owner["postcode"] + " " + owner["city"])
            person_street.append(owner["street"] + " " + owner["housenumber"])
            telefon.append(owner["linenumber"])
            mobile.append(owner["mobil"])
            email.append(owner["email"])

    return pd.DataFrame(data={
        "KLSID": klsid,
        "Ort": ort,
        "Strasse": strasse,
        "Rolle": rolle,
        "Name": name,
        "Ort2": person_ort,
        "Strasse2": person_street,
        "Telefon": telefon,
        "Mobile": mobile,
        "email": email
        })

In [6]:
import pandas as pd
import os
import glob
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from pathlib import Path
chrome_options = Options()
chrome_options.add_argument("--headless")

In [7]:
roots_list = [
"/Users/dlprojectsit/Library/CloudStorage/OneDrive-SharedLibraries-DLProjectsGmbH/Data Management DL Projects - BAU/RV-07 Dresden/BVH-01 Dresden Cotta/Cotta West/Baupläne (HK+NVT)"
, "/Users/dlprojectsit/Library/CloudStorage/OneDrive-SharedLibraries-DLProjectsGmbH/Data Management DL Projects - BAU/RV-07 Dresden/BVH-01 Dresden Cotta/Cotta Ost/Baupläne (HK+NVT)"
]

In [8]:
nvt_path_list = [Path(path) for root in roots_list for path in glob.glob(root + "/*/*") if "NVT" in Path(path).stem]
data_dict = {}
for nvt_path in nvt_path_list:
    # hk_number = nvt_path.parent.stem.split("+")[0].replace("HK ", "").replace(" ", "") # There is no need to it
    nvt_number = nvt_path.stem.replace("NVT ","")
    data_dict[nvt_number] = {"path": str(nvt_path),
                             "place": "Dresden", # must be taken from the folders later
                             "nvt_telekom_list": []
                            }
# data_dict

# Scrapping code

In [9]:
URL = "https://glasfaser.telekom.de/auftragnehmerportal-ui/home?a-cid=50708"

In [10]:
# log in code
browser = Chrome(options = chrome_options)
browser.get(URL)
browser.find_element('id', 'username').send_keys('ertugrul.yilmaz@dl-projects.de')
browser.find_element('id', 'password').send_keys('Ertu2022!')
browser.find_element('name', 'login').click()

In [11]:
# Now, moving to the search page
browser.get("https://glasfaser.telekom.de/auftragnehmerportal-ui/property/search")

In [12]:
def click_the_search_button(browser):
    search_button_element = browser.find_element('id', 'searchCriteriaForm:searchButton')
    search_button_element.click()

In [13]:
def click_export_excel_button(browser):
    export_excel_element = browser.find_element('id', 'searchResultForm:propertySearchSRT:exportPropertiesData')
    export_excel_element.click()

In [14]:
def click_reset_filter_button(browser):
    reset_button = browser.find_element('id', 'searchCriteriaForm:resetButton')
    reset_button.click()


In [15]:
#### First filter
time.sleep(2)
# Get the select of the filter value directly and click on it!
gap_installation_option = browser.find_element('xpath', '//option[@value="GFAP_INSTALLATION"]')
# gap_installation_option.click()
browser.execute_script("arguments[0].click();", gap_installation_option);
time.sleep(3)

In [16]:
# Filterning table result functions
def filter_according_to_nvt_number(browser, nvt_number):
    print(nvt_number)
    nvt_li = browser.find_element('xpath', '//label[contains(text(), "{}")]/parent::li'.format(nvt_number))
    nvt_checkbox = nvt_li.find_element('css selector', 'input')
    browser.execute_script("arguments[0].click();", nvt_checkbox);
    return nvt_li

def filter_according_to_nvt_list(browser, nvt_list):
    for nvt_number in nvt_list:
        filter_according_to_nvt_number(browser, nvt_number)
        time.sleep(2)

In [17]:
nvt_container = browser.find_element('id', 'searchCriteriaForm:nvtArea_panel')
nvt_list_from_telekom = [x.find_element('css selector', 'label').get_attribute("innerHTML") for x in nvt_container.find_elements('css selector', 'li')]

In [18]:
# Now matching:
for our_nvt in data_dict.keys():
    for nvt_from_telekom in nvt_list_from_telekom:
        if our_nvt in nvt_from_telekom:
            data_dict[our_nvt]["nvt_telekom_list"].append(nvt_from_telekom)

In [19]:
# data_dict

In [20]:
# click_the_search_button(browser)

In [21]:
def get_next_page_button(index):
    next_page_button = browser.find_elements("xpath", '//a[@aria-label="Page {}"]'.format(index))
    if len(next_page_button) > 0:
        return next_page_button[0]
    return None

In [22]:
# data_dict

In [23]:
for our_nvt in data_dict.keys():
    filter_according_to_nvt_list(browser, data_dict[our_nvt]["nvt_telekom_list"])
    time.sleep(5)
    click_the_search_button(browser)
    time.sleep(5)
    data_dict[our_nvt]["kls_dicts"] = {}
    for i in range(1, 100):
        # function: log_number_of_eyes_of_the_current_page:
        eys_links = browser.find_elements("xpath", '//a[contains(@id,":viewSelectedRowItem")]')
        print("Number of rows is ", len(eys_links), " in Page ", str(i))
        
        kls_dicts = get_eyes_data(browser)
        data_dict[our_nvt]["kls_dicts"] = {**data_dict[our_nvt]["kls_dicts"], **kls_dicts} 
        
        next_page_button = get_next_page_button(i + 1)
        if next_page_button == None:
            break
        else:
            browser.execute_script("arguments[0].click();", next_page_button);
            time.sleep(5)
    
    log("Exporting excel file to: " + str(data_dict[our_nvt]['path']))
    # df = export_kls_to_excel(data_dict[our_nvt]["kls_dicts"])
    # df.to_excel(str(data_dict[our_nvt]['path']) + "/" + "generated_anshprechpartner_list.xlsx", engine='xlsxwriter')  
    click_reset_filter_button(browser)
    time.sleep(5)
    print("________________________")
    #click_export_excel_button(browser)

42V1016
Number of rows is  8  in Page  1
Log: Navigating to address page
Log: Starting with kls: 16213699
Log: Navigating to contact tab
There is not contact people!
Log: Navigating to owner tab
Reading owner:  {'name': 'MRS Frauke Craco', 'email': 'info@kretzschmar-partner.de', 'mobil': '', 'linenumber': '0351436200', 'postcode': '01219', 'city': 'Dresden', 'street': 'Goppelner Str.', 'housenumber': '40', 'decisionmaker': 'true'}
Log: Pressing close button of address page
Log: Navigating to address page
Log: Starting with kls: 16213713
Log: Navigating to contact tab


KeyboardInterrupt: 

In [24]:
# import json
# with open('data.json', 'w') as f:
#     json.dump(data_dict, f)
